In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('/data/data/UserMovie_train.txt',encoding='gb2312',delim_whitespace=True,index_col=None)

In [3]:
data.head()

,uid,mid,timeStamp,star
0,1722994,1306505,2007-08-22,5.0
1,1405477,10574468,2013-04-24,4.0
2,15849871,4910186,2011-12-20,5.0
3,1068524,1304643,2006-02-07,3.0
4,1307041,1851857,2008-09-22,4.0


In [4]:
data['timeStamp'] = data['timeStamp'].apply(lambda x:str(x)[0:4])
data.head()

,uid,mid,timeStamp,star
0,1722994,1306505,2007,5.0
1,1405477,10574468,2013,4.0
2,15849871,4910186,2011,5.0
3,1068524,1304643,2006,3.0
4,1307041,1851857,2008,4.0


In [5]:
def hash_star(x):
    if x>=3:
        return 1
    else:
        return 0

data['star'] = data['star'].apply(lambda x:hash_star(x))

In [3]:
# 目前不需要
all1 = pd.read_csv('libsvm.txt',sep=' ',encoding='GB2312')
def hash_id(x):
    return int(str(x)[2:])

def hash_star(x):
    if x>=3:
        return 1
    else:
        return 0

all1.columns = ['star','uid','mid','timeStamp']    
for i in ['uid','mid']:
    all1[i] = all1[i].apply(lambda x:hash_id(x))
    
all1['timeStamp'] = all1['timeStamp'].apply(lambda x:str(x)[2:])
    
all1['star'] = all1['star'].apply(lambda x:hash_star(x))

In [6]:
all1=data

In [7]:
all1.head(5)

,uid,mid,timeStamp,star
0,1722994,1306505,2007,1
1,1405477,10574468,2013,1
2,15849871,4910186,2011,1
3,1068524,1304643,2006,1
4,1307041,1851857,2008,1


In [8]:
temp = all1.groupby(['uid']).size().reset_index().rename(columns={0:'times'})
temp.head()

,uid,times
0,1000001,158
1,1000002,33
2,1000023,243
3,1000028,78
4,1000030,190


In [9]:
save_uid = temp[temp['times']<=6000]['uid'].values
print('TEMP UID: ',temp.shape)
print('ALL UID: ',all1.shape)

TEMP UID:  (48765, 2)
ALL UID:  (8777260, 4)


In [10]:
temp = temp[temp['uid'].isin(save_uid)]
all1 = all1[all1['uid'].isin(save_uid)]
print('DELETE ID: ',all1.shape)

DELETE ID:  (8434377, 4)


In [11]:
all_movie = list(set(all1['mid'].values))

In [12]:
import random
random.randint(1,len(all_movie))

52268

In [16]:
import random
import time
imposstive = {
    'uid' : [],
    'mid' : [],
    'star' : [],
    'timeStamp' : []
}

In [17]:
all_movie = list(set(all1['mid'].values))
tot = 0
t1 = time.time()

In [21]:
temp.head()

,uid,times
0,1000001,158
1,1000002,33
2,1000023,243
3,1000028,78
4,1000030,190


In [22]:
for i,j in temp[['uid','times']].values:
    tot+=1
    kfc = random.sample(all_movie,int(j*4)) # 每次从 all_movie 中随机选 times * 4 个
    #print(type(kfc),len(kfc))
    for k in all1[all1['uid']==i]['mid'].unique():
        try:
            kfc.remove(k)
        except:
            continue
    for k in range(j*3):
        imposstive['uid'].append(i)
        imposstive['star'].append(0)
        kkk = random.randint(1,len(kfc)-1)
        imposstive['mid'].append(kfc[kkk])
        imposstive['timeStamp'].append(random.randint(2005,2016))
        try:
            kfc.remove(all_movie[kkk])
        except:
            continue
    if tot % 1000 == 0:
        print(tot)
        print(time.time()-t1)
        t1 = time.time()

1000
838.171370267868
2000
129.26871347427368
3000
125.66919660568237
4000
141.36979126930237
5000
64.53927397727966
6000
144.33055305480957
7000
85.58274531364441
8000
69.23642921447754
9000
124.8469569683075
10000
120.25094699859619
11000
88.68109774589539
12000
83.45695281028748
13000
61.8263213634491
14000
61.5665009021759
15000
55.861602783203125
16000
57.06078100204468
17000
68.83034110069275
18000
43.61061882972717
19000
75.4701201915741
20000
55.602004289627075
21000
88.02857637405396
22000
110.60069847106934
23000
45.88619899749756
24000
98.06439661979675
25000
82.86839485168457
26000
62.1473753452301
27000
75.37487268447876
28000
48.554770946502686
29000
109.44832348823547
32000
61.449716567993164
33000
89.11757636070251
34000
49.75818395614624
35000
68.72098803520203
36000
60.3069806098938
37000
134.82549595832825
38000
104.049724817276
39000
82.31048011779785
40000
64.33726453781128
41000
52.53850793838501
42000
74.05622601509094
43000
64.32852935791016
44000
49.36347794532

In [33]:
from pandas import DataFrame as DF
all_data2 = DF(imposstive)

In [36]:
all_data2.head()

,uid,mid,star,timeStamp
0,1000001,21371640,0,2015
1,1000001,4158024,0,2013
2,1000001,2056602,0,2007
3,1000001,3788025,0,2008
4,1000001,4116987,0,2006


In [38]:
feature_name = all_data2.columns

In [39]:
final_data = pd.concat([all1[feature_name],all_data2[feature_name]],axis=0).reset_index(drop=True)
final_data.to_csv('final_data1.csv',index=False)

In [ ]:
final_data.info()

In [40]:
final_data['new_mid'] = final_data['mid'].map(lambda x:int(x)) #lambda x:str(x)[2:]

In [41]:
final_data.head()

,uid,mid,star,timeStamp,new_mid
0,1722994,1306505,1,2007,1306505
1,1405477,10574468,1,2013,10574468
2,15849871,4910186,1,2011,4910186
3,1068524,1304643,1,2006,1304643
4,1307041,1851857,1,2008,1851857


In [42]:
final_data['new_mid'].max()

24714535

In [ ]:
all_data2.head(10)

In [ ]:
all1_temp = all1[['uid','mid','star','timeStamp']].copy()